<a href="https://colab.research.google.com/github/samuelfernandof/LLM-NLP-Projects/blob/main/Wav2Vec2_PT_BR_finetuned_automatic_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers==4.30.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:00


In [ ]:
!pip install -q torch==2.0.1

https://pypi.org/project/soundfile/

In [ ]:
!pip install -q soundfile

https://pypi.org/project/librosa/

In [4]:
!pip install -q librosa

https://pypi.org/project/datasets/

In [5]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [6]:
# Imports
import torch
import librosa
import transformers
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import warnings
warnings.filterwarnings('ignore')

## Modelo Facebook Wav2Vec2

Este é um modelo básico pré-treinado e ajustado em 960 horas de Librispeech em áudio de fala amostrado de 16kHz. Ao usar o modelo, certifique-se de que sua entrada de fala também seja amostrada em 16Khz.

https://arxiv.org/abs/2006.11477

https://huggingface.co/facebook/wav2vec2-base-960h

In [ ]:
# Carrega o tokenizer e o modelo
tokenizador_1 = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
modelo_1 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

https://huggingface.co/datasets/patrickvonplaten/librispeech_asr_dummy/tree/main

In [ ]:
# Carrega conjunto de dados fictício de arquivos de áudio
dados = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split = "validation")

In [10]:
dados

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [ ]:
dados['file']

In [ ]:
dados['audio']

In [14]:
# Aplica o tokenizador em 1 arquivo de áudio
input_values = tokenizador_1(dados[0]["audio"]["array"],
                             return_tensors = "pt",
                             padding = "longest",
                             sampling_rate = 16_000).input_values

In [16]:
input_values[0]

tensor([-0.0048, -0.0018, -0.0135,  ...,  0.0077,  0.0089,  0.0083])

In [17]:
logits = modelo_1(input_values).logits

In [18]:
# Argmax com os maiores logits para extrair as previsões
predicted_ids = torch.argmax(logits, dim = -1)

In [19]:
# Previsão de texto do arquivo de áudio
predicted_ids

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  4,  4,  0, 17,  0,
          0,  0,  0,  7,  0,  0,  9,  0,  0,  0,  4,  4,  4,  0,  0,  0,  0, 12,
          0,  0,  0,  7, 10,  0, 14, 14,  4,  4,  6,  6,  0,  8,  0,  4,  4,  6,
         11,  5,  5,  4,  4,  4,  0, 16, 16,  0,  9,  9,  0, 10,  0,  0, 25,  0,
          0,  0,  5, 13, 13,  0,  0,  0, 12,  5,  5,  4,  4,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12,  0,
          0,  0,  0, 10,  0, 13, 13,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  4,
          0,  0,  0,  5, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0, 12,
          0,  0,  6,  0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,

In [20]:
# Tokeniza a previsão
transcription = tokenizador_1.batch_decode(predicted_ids)

In [21]:
transcription

['A MAN SAID TO THE UNIVERSE SIR I EXIST']

## Trabalhando com Áudio em Português

https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-portuguese

In [22]:
# Modelo
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-portuguese"

In [27]:
# Carrega o tokenizer e o modelo
tokenizer_2 = Wav2Vec2Processor.from_pretrained(MODEL_ID)
modelo_2 = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [28]:
# Idioma dos áudios
LANG_ID = "pt"

In [29]:
# Número de amostras
SAMPLES = 10

In [ ]:
!pip install --upgrade datasets
!pip install --upgrade tqdm

In [35]:
from datasets import load_dataset

LANG_ID = "en"  # ou outro identificador de linguagem apropriado
SAMPLES = 100  # defina um número menor de amostras para teste

try:
    dados_teste = load_dataset("common_voice", LANG_ID, split=f"test[:{SAMPLES}]")
    print("Dataset carregado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Generating train split:   0%|          | 0/564337 [00:00<?, ? examples/s]

Ocorreu um erro: An error occurred while generating the dataset


In [ ]:
dados_teste

In [ ]:
dados_teste['path']

In [37]:
# Função que processa os arquivos de áudio e carregando como arrays
def processa_audio(batch):

    # Extrai array e sampling_rate dos arquivos de áudio
    speech_array, sampling_rate = librosa.load(batch["path"], sr = 16_000)

    # Array do áudio
    batch["speech"] = speech_array

    # Texto (label)
    batch["sentence"] = batch["sentence"].upper()

    return batch

In [ ]:
# Aplica a função
dados_teste = dados_teste.map(processa_audio)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
dados_teste

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech'],
    num_rows: 10
})

In [ ]:
dados_teste["speech"]

[[3.637978807091713e-12,
  -6.366462912410498e-12,
  1.8189894035458565e-12,
  3.637978807091713e-12,
  1.4551915228366852e-11,
  -1.0913936421275139e-11,
  1.0913936421275139e-11,
  -3.637978807091713e-12,
  1.8189894035458565e-11,
  0.0,
  -2.7284841053187847e-12,
  2.546585164964199e-11,
  -2.3646862246096134e-11,
  -1.8189894035458565e-11,
  7.275957614183426e-12,
  1.0913936421275139e-11,
  -3.637978807091713e-12,
  -2.1827872842550278e-11,
  9.094947017729282e-13,
  -2.4556356947869062e-11,
  1.546140993013978e-11,
  -1.546140993013978e-11,
  2.1827872842550278e-11,
  -9.094947017729282e-12,
  -3.637978807091713e-12,
  -2.2737367544323206e-12,
  -1.0913936421275139e-11,
  -1.8189894035458565e-12,
  -1.4551915228366852e-11,
  2.1827872842550278e-11,
  -3.637978807091713e-12,
  2.9103830456733704e-11,
  7.275957614183426e-12,
  -3.637978807091713e-12,
  -1.8189894035458565e-12,
  1.864464138634503e-11,
  0.0,
  3.637978807091713e-12,
  -7.275957614183426e-12,
  -1.4551915228366852e

In [ ]:
# Aplica o tokenizador
inputs = tokenizer_2(dados_teste["speech"], return_tensors = "pt", padding = True, sampling_rate = 16_000)

In [ ]:
# Extrai os logits (previsões do modelo)
with torch.no_grad():
    logits = modelo_2(inputs.input_values, attention_mask = inputs.attention_mask).logits

In [ ]:
# Previsões
predicted_ids = torch.argmax(logits, dim=-1)

In [ ]:
# Tokenizer das previsões
predicted_sentences = tokenizer_2.batch_decode(predicted_ids)

In [ ]:
# Imprime valor real e valor previsto pelo modelo
for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Valor Real:", dados_teste[i]["sentence"])
    print("Previsão (Texto Extraído do Áudio):", predicted_sentence)

----------------------------------------------------------------------------------------------------
Valor Real: ESTAMOS AQUI PARA PEDIR EMPRESTADO
Previsão (Texto Extraído do Áudio): estamos aqui para pedir emprestado
----------------------------------------------------------------------------------------------------
Valor Real: GRAÇAS A DEUS VOCÊ ESTÁ AQUI!
Previsão (Texto Extraído do Áudio): graças a deus você está aqui
----------------------------------------------------------------------------------------------------
Valor Real: PRECISAMOS NOS APRESSAR.
Previsão (Texto Extraído do Áudio): precisamos nos apressar
----------------------------------------------------------------------------------------------------
Valor Real: A NECESSIDADE É O PAI NA INOVAÇÃO.
Previsão (Texto Extraído do Áudio): a necessidade é pai na inovação
----------------------------------------------------------------------------------------------------
Valor Real: VOCÊ PODERIA TER MORRIDO DEPOIS QUE A PAZ FOSS

# Fim